# This is going to be a 2D attempt at making the serotonin transport thing work

In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from collections import namedtuple

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
concentration = np.array([[5,0,0,0,0]])
concentration.shape

(1, 5)

Okay, what I'm trying to do is set up euler's method for each of these different ways

We have the following modes of transport:
* Diffusion
* Adsorbtion 
* Reaction

The overall mass balance is represented by the following equation:

$ \frac{\partial}{\partial t}(\rho c_i) + \frac{1}{r}\frac{\partial}{\partial r}(r \rho v_r c_i) + \frac{\partial}{\partial z}(\rho v_z c_i) = \frac{1}{r} \frac{\partial}{\partial r}(r \rho D \frac{\partial c_i}{\partial r}) + \frac{\partial}{\partial z}(\rho D \frac{\partial c_i}{\partial z}) + R_i$

We can make the following assumptions:

* No accumulation
* No change in density
* No velocity in the $r$ direction
* No change in velocity in the $z$ direction

Therefore the differiential equation expands as follows:

$ \rho v_z \frac{\partial c_i}{\partial z} = \rho D \frac{1}{r} (r \frac{\partial^2 c_i}{\partial r^2} + \frac{\partial c_i}{\partial r}) + \rho D \frac{\partial^2 c_i}{\partial z^2} + R_i $

For the previous equation we have a variety of ways to approximate each derivative:

$ \frac{ \partial y } {\partial x} = \frac{y_{n} - y_{n-1}}{\Delta x}$

$ \frac{ \partial y } {\partial x} = \frac{y_{n+1} - y_{n-1}}{2 \Delta x}$

$ \frac{ \partial y } {\partial x} = \frac{y_{n-2} - 8 y_{n-1} - + 8 y_{n+1} - y_{n+2}}{12 \Delta x}$

$\frac{ \partial^2 y } {\partial x^2} = \frac{y_{n+1} - 2 y_n + y_{n-1}}{(\Delta x)^2}$




# Starting with Abdul

$ \rho {v_z} \frac{\partial C}{\partial z} = \rho D \frac{1}{r} (r \frac{\partial^2 C}{\partial r^2} + \frac{\partial C}{\partial r})+ \rho D \frac{\partial^2 C}{\partial z^2} \longrightarrow  $ Assuming no reaction

$- \rho D \frac{\partial^2 C}{\partial z^2} + \rho v_z \frac{\partial C}{\partial z} = \rho D \frac{1}{r} (r \frac{\partial^2 C}{\partial r^2} + \frac{\partial C}{\partial r})$

$ -\frac{\partial^2 C}{\partial z^2} + \frac{v_z}{D} \frac{\partial C}{\partial z} = \frac{\partial^2 C}{\partial r^2} + \frac{1}{r} \frac{\partial C}{\partial r} $

Let $ C = f(z)g(r) $

$ -\frac{\partial^2}{\partial z^2}(f*g) + \frac{v_z}{D} \frac{\partial}{\partial z} (f*g) = \frac{\partial^2}{\partial r^2} (f*g) + \frac{1}{r} \frac{\partial}{\partial r} (f*g) $

$ -\frac{\partial^2 f}{\partial z^z}g + \frac{v_z}{D} \frac{\partial f}{\partial z}g = \frac{\partial^2 g}{\partial r^2}f + \frac{1}{r}\frac{\partial g}{\partial r} f $

$ (-\frac{\partial^2 f}{\partial z^z} + \frac{v_z}{D} \frac{\partial f}{\partial z})g = (\frac{\partial^2 g}{\partial r^2} + \frac{1}{r}\frac{\partial g}{\partial r}) f $

$ (-\frac{\partial^2 f}{\partial z^z} + \frac{v_z}{D} \frac{\partial f}{\partial z})\frac{1}{f} = (\frac{\partial^2 g}{\partial r^2} + \frac{1}{r}\frac{\partial g}{\partial r})\frac{1}{g} = -\lambda $

$ \rightarrow -\frac{\partial^2 f}{\partial z^2} + \frac{v_z}{D} \frac{\partial f}{\partial z} = -\lambda f $

$ \frac{\partial^2 f}{\partial z^2} - \frac{v_z}{D} \frac{\partial f}{\partial z} - \lambda f = 0 $

$ r^2 - \frac{v_z}{D} r - \lambda = 0 $

$ r = \frac{\frac{v_z}{D} \pm \sqrt{(\frac{v_z}{D})^2 + 4 \lambda}}{2} $

$ f = C_1 e^{\lgroup \frac{v_z}{D}+\sqrt{(\frac{v_z}{D})^2 + 4\lambda}\rgroup \frac{z}{2}} + C_2 e^{\lgroup \frac{v_z}{D}-\sqrt{(\frac{v_z}{D})^2 + 4\lambda}\rgroup \frac{z}{2}} $

$ \rightarrow \frac{\partial ^2 g}{\partial r^2} + \frac{1}{r} \frac{\partial g}{\partial r} = -\lambda g $

$ \frac{\partial^2 g}{\partial r^2} + \frac{1}{r}\frac{\partial g}{\partial r} + \lambda g = 0 $

$ g = C_3 J_o (\sqrt{\lambda} r) + C_4 Y_o (\sqrt{\lambda} r) $

$ \therefore C = (C_1 e^{\lgroup \frac{v_z}{D}+\sqrt{(\frac{v_z}{D})^2 + 4\lambda}\rgroup \frac{z}{2}} + C_2 e^{\lgroup \frac{v_z}{D}-\sqrt{(\frac{v_z}{D})^2 + 4\lambda}\rgroup \frac{z}{2}})(C_3 J_o (\sqrt{\lambda} r) + C_4 Y_o (\sqrt{\lambda} r)) $

In [2]:
import scipy
import numpy as np
import pandas as pd

In [3]:
trypConditionList = [(.1,5.386e-8,6.44e-4),(.5,5.386e-8,6.44e-4),(1,5.386e-8,6.44e-4)] #Concentration, Diffusivity, Permeability

radius = 2.5/2/100
length = 7.5
max_velocity = .0287/60
timestep = 20/3600
rings = 20
sections = 500

In [4]:
conc, D, _ = condition = trypConditionList[-1]

In [17]:
intset = np.zeros([500, 20])

In [20]:
intset[0] = conc
intset[0]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.])